## Script to perform NLTE correction using MPIA
Some notes: mechanicalsoup is required to be installed in order to run this.

This script uses data obtained from the **MPIA NLTE Correction** tool (nlte.mpia.de) from:

 **O**:
 - Sitnova, Mashonkina, Ryabchikova 2013, AstL, 39, 126S
 
 **Mg (IR)**:
 - Bergemann, Kudritzki, Gazak et al. 2015, ApJ, 804, 113
 
 **Mg (optical)**:
 - Bergemann, Collet, Amarsi et al. 2017, ApJ, 847, 15
 
 **Si**:
 - Bergemann, Kudritzki, Plez et al. 2013, ApJ, 764, 115
 
 **Ca**:
 - Mashonkina, L.; Korn, A. J.; Przybilla, N. 2007, A&A, 461, 261M
 
 **Ti**:
 - Bergemann 2011, MNRAS, 413, 2184

**Cr**: 
- Bergemann, M., Cescutti, G. A&A, 522, A9, 13 pp. 

 **Mn**:
 - Bergemann & Gehren 2008, A&A, 492, 823
 
 **Fe, Ti (IR)**: 
 - Bergemann, Kudritzki, Plez et al. 2012, ApJ, 751, 156
 
 **Fe (optical)**:
 - Bergemann, Lind, Collet et al. 2012, MNRAS, 427, 1
 
 **Co**:
 - Bergemann, Pickering, & Gehren 2010, MNRAS, 401, 1334


In [ ]:
# If mechanical soup isn't installed, install either here, or via your preferred method
# pip install mechanicalsoup

In [1]:
import mechanicalsoup
import numpy as np
import pandas as pd

In [31]:
#           ['O', 'Mg', 'Si', 'CaI', 'CaII', 'TiI', 'TiII', 'Cr', 'Mn', 'FeI', 'FeII', 'Co']
elem_list = [8.01, 12.01, 14.01, 20.01, 20.02, 22.01, 22.02, 24.01, 25.01, 26.01, 26.02, 27.01]

In [80]:
def nlte_cor(elem, lines, t, g, feh, vt):
    """
    DESCRIPTION --------------------------------------------------------
        Given input parameters, return the NLTE correction performed by MPIA
        
    PARAMETERS ---------------------------------------------------------
        elem (float) - a chemical element in the form of the contents of elem_list,
            defined in the previous code-block
        lines (list(float)) - a list of the lines at which you want the NLTE correction
            to be performed for this element
        t (float) - the effective temperature of your star
        g (float) - the surface gravity of your star
        feh (float) - the metallicity [Fe/H] of your star
        vt (float) - the micro-turbulent velocity of your star
        
    RETURNS -------------------------------------------------------------
        np.array() - a 2D numpy array with 3 elements per row, such that each row
            has the element id, line at which the correction was performed, and
            the resulting correction delta
            [elem, line, delta]
            
            If the correction was not able to be performed, delta is replaced with the
            value '999'
    """
    # If the element isn't in the element list, return 999 for delta
    if elem not in elem_list:
        print('Element not available: ' + str(elem))
        delta = np.full(len(lines), 999.0)
        return np.transpose(np.array([np.full(len(lines), elem), lines, delta]))

    # If the element is either O, Mn, or Co, the NLTE correction must be done at
    #  a separate webpage
    if elem in [8.01, 25.01, 27.01]:
        weblink = 'https://nlte.mpia.de/gui-siuAC_secEnew.php'
    else:
        weblink = "https://nlte.mpia.de/gui-siuAC_secE.php"
    
    browser = mechanicalsoup.StatefulBrowser()
    browser.open(weblink)
    
    form = browser.select_form()
    
    param_txt = "{} {} {} {} {}".format('my_star', t, g, feh, vt)
    form.set_textarea({"user_input":param_txt})
    
    line_txt = ""
    
    for line in lines:
        append = "{} {}\n".format(line, elem)
        line_txt += append
        
    form.set_textarea({"lines_input":line_txt})
    
    browser.submit_selected()
        
    result_txt = browser.page.get_text().split('lines(A)')[-1].split('Download')[0].split('my_star')
    
    lines = result_txt[0].split()
    delta = result_txt[1].split()
        
    for i, d in enumerate(delta):
        if float(d) == 30.0:
            delta[i] = 999.0
            print('Line not in linelist (or too weak) at: ' + str(elem) + ' ' + str(lines[i]))
        elif float(d) == 20.0:
            delta[i] = 999.0
            print('NLTE not converged at: ' + str(elem) + ' ' + str(lines[i]))
        elif float(d) == 10.0:
            delta[i] = 999.0
            print('Error in lineformation' + str(elem) + ' ' + str(lines[i]))
    # This is a fail-safe, feel free to comment out the below condition if you do not want it
        elif float(d) == 0.0:
            delta[i] = 999.0
            print('No NLTE departures for this line' + str(elem) + ' ' + str(lines[i]))


    return np.transpose(np.array([np.full(len(lines), elem), lines, delta]))

## Example of one element usage:

In [55]:
test_lines = [5379.574, 3125.680, 8179.030]

In [56]:
nlte_cor(26.01, test_lines, 6491, 4.44, -3.16, 1.11)

Line not in linelist at: 26.01 5379.574
Line not in linelist at: 26.01 8179.03


array([['26.01', '5379.574', '999'],
       ['26.01', '3125.68', '0.139'],
       ['26.01', '8179.03', '999']], dtype='<U32')

### For easy use, input data should follow a convention similar to:
```python
df = pd.DataFrame(data_array)
df.columns = ['elem', 'line']
star_info = ['T', 'log(g)', '[Fe/H]', 'vt'] # Can import directly from the atmosphere params csv
```

#### To quickly convert a numeric id to the string required to run the script, using pandas, I find that this following method works great:

```python
elem_dict = {8:8.01, 12:12.01, 14:14.01, 20:20.01, 20.1:20.02, 22:22.01, 22.1:22.02, 24:24.01, 25:25.01, 26:26.01, 26.1:26.02, 27:27.01}
df[['elem']] = df[['elem']].replace(elem_dict)
```

#### And this is a quick way to run multiple lines into the script
```python
df = df.sort_values('elem')

grouped = df.groupby('elem')

results = []

for elem in df.elem.unique():
    linelist = grouped.get_group(elem).to_numpy()[:,0]
    results.append(nlte_cor(elem, linelist, *star_info))
    print(elem)
    
results = np.concatenate(results)

df = pd.DataFrame(results)
df.columns = ['elem', 'line', 'NLTE delta']
```

## Example of multi-element usage:

In [83]:
elem = [3, 12, 11, 26.1, 26.1, 9, 5, 22, 22, 22.1]
line = [6103.6, 5172.68, 5688.2, 3815.84, 3565.380, 5638.2, 8392.9, 5702.67, 7362.93, 6092.798]

df = pd.DataFrame(np.transpose([elem, line]))
df.columns = ['elem', 'line']
star_info = [6400, 3, -3, 1.5]

elem_dict = {8:8.01, 12:12.01, 14:14.01, 20:20.01, 20.1:20.02, 22:22.01, 22.1:22.02, 24:24.01, 25:25.01, 26:26.01, 26.1:26.02, 27:27.01}
df[['elem']] = df[['elem']].replace(elem_dict)

df = df.sort_values('elem')

In [84]:
grouped = df.groupby('elem')

results = []

for elem in df.elem.unique():
    linelist = grouped.get_group(elem).to_numpy()[:,1] # Make sure that this returns the lines!!!
    results.append(nlte_cor(elem, linelist, *star_info))
    print(elem)
    
results = np.concatenate(results)

Element not available: 3.0
3.0
Element not available: 5.0
5.0
Element not available: 9.0
9.0
Element not available: 11.0
11.0
12.01
Line not in linelist (or too weak) at: 22.01 5702.67
Line not in linelist (or too weak) at: 22.01 7362.93
22.01
Line not in linelist (or too weak) at: 22.02 6092.798
22.02
Line not in linelist (or too weak) at: 26.02 3815.84
Line not in linelist (or too weak) at: 26.02 3565.38
26.02


In [90]:
df = pd.DataFrame(results)
df.columns = ['elem', 'line', 'delta']
df

,elem,line,delta
0,3.0,6103.6,999.0
1,5.0,8392.9,999.0
2,9.0,5638.2,999.0
3,11.0,5688.2,999.0
4,12.01,5172.68,0.159
5,22.01,5702.67,999.0
6,22.01,7362.93,999.0
7,22.02,6092.798,999.0
8,26.02,3815.84,999.0
9,26.02,3565.38,999.0
